<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

In [22]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
outPath = "../data/vegFracControls.csv"

## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [23]:
driver_paths = [os.path.join(dp, f) for (dp, _, fn) in os.walk("../data/driving_Data/") for f in fn if f.endswith('.nc')]

driver_names = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths]

file_table = pd.DataFrame({'filepath': driver_paths, 'file_name': driver_names})
file_table

,filepath,file_name
0,../data/driving_Data/BurntArea.nc,BurntArea
1,../data/driving_Data/crop.nc,crop
2,../data/driving_Data/MaxWind.nc,MaxWind
3,../data/driving_Data/buffalo.nc,buffalo
4,../data/driving_Data/cattle.nc,cattle
5,../data/driving_Data/MADD_CMORPH.nc,MADD_CMORPH
6,../data/driving_Data/MADD_CRU.nc,MADD_CRU
7,../data/driving_Data/MADD_GPCC.nc,MADD_GPCC
8,../data/driving_Data/MADD_MSWEP.nc,MADD_MSWEP
9,../data/driving_Data/MADM_CMORPH.nc,MADM_CMORPH


Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [24]:
from pdb import set_trace as browser
def nc_extract(fpath):
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['layer'][:,:])
        gflat = gdata.flatten()
        #browser()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [25]:
values = [nc_extract(dp) for dp in driver_paths]
for i in values: print(i.shape)

Processing: ../data/driving_Data/BurntArea.nc
Processing: ../data/driving_Data/crop.nc
Processing: ../data/driving_Data/MaxWind.nc
Processing: ../data/driving_Data/buffalo.nc
Processing: ../data/driving_Data/cattle.nc
Processing: ../data/driving_Data/MADD_CMORPH.nc
Processing: ../data/driving_Data/MADD_CRU.nc
Processing: ../data/driving_Data/MADD_GPCC.nc
Processing: ../data/driving_Data/MADD_MSWEP.nc
Processing: ../data/driving_Data/MADM_CMORPH.nc
Processing: ../data/driving_Data/MADM_CRU.nc
Processing: ../data/driving_Data/MADM_GPCC.nc
Processing: ../data/driving_Data/MADM_MSWEP.nc
Processing: ../data/driving_Data/goat.nc
Processing: ../data/driving_Data/MAP_CMORPH.nc
Processing: ../data/driving_Data/MAP_CRU.nc
Processing: ../data/driving_Data/MAP_GPCC.nc
Processing: ../data/driving_Data/MAP_MSWEP.nc
Processing: ../data/driving_Data/MAT.nc
Processing: ../data/driving_Data/MConc_CMORPH.nc
Processing: ../data/driving_Data/MConc_CRU.nc
Processing: ../data/driving_Data/MConc_GPCC.nc
Proce

Turn this into a dataframe for the analysis.

In [26]:
# turn list into a dataframe
fire_df = pd.DataFrame(np.array(values).T, columns=driver_names)

# replace null flags with pandas null
fire_df.replace(fire_df["BurntArea"][0], np.nan, inplace=True)

# drop all null rows (are ocean and not needed in optim)
fire_df.dropna(inplace=True)

Check that we've built it correctly.

In [27]:
fire_df.head()


,BurntArea,crop,MaxWind,buffalo,cattle,MADD_CMORPH,MADD_CRU,MADD_GPCC,MADD_MSWEP,MADM_CMORPH,...,sheep,MTCM,BurntArea_GFED,BurntArea_GFED,BurntArea_meris,BurntArea_MODIS,BurntArea_MCD,BurntArea_GFED_four_s,BurntArea_GFED_four,BurntArea_MCD_forty_five
108,0.000103,0.000000,0.000000,0.0,3.666429e-02,0.937685,1.0,0.954822,0.838237,0.906849,...,0.000000e+00,7.064286,0.000103,0.000101,0.0,0.0,0.000101,0.000103,0.000101,0.000101
109,0.000038,0.000007,6.386633,0.0,3.002909e-01,0.949989,1.0,0.965384,0.876759,0.988337,...,3.747991e-03,5.500000,0.000038,0.000000,0.0,0.0,0.000101,0.000038,0.000000,0.000101
110,0.000006,0.000002,6.260526,0.0,4.050972e-01,0.907838,1.0,0.954631,0.866844,0.901959,...,5.621986e-03,5.735714,0.000006,0.000051,0.0,0.0,0.000004,0.000006,0.000051,0.000004
111,0.000008,0.000014,5.843527,0.0,2.529809e-01,0.857775,1.0,0.949744,0.874905,0.729050,...,4.868020e-03,6.807143,0.000008,0.000000,0.0,0.0,0.000015,0.000008,0.000000,0.000015
112,0.000000,0.000000,0.000000,0.0,3.478891e-14,0.892677,1.0,0.939389,0.894754,0.812315,...,6.317959e-16,7.907143,0.000000,0.000000,0.0,0.0,0.000036,0.000000,0.000000,0.000036


Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [28]:
savepath = os.path.expanduser(outPath)
fire_df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>